In [ ]:
import datetime
import itertools
import os
import re
import requests
import sys
import time
from copy import deepcopy
from collections import namedtuple, defaultdict

from bs4 import BeautifulSoup as BS
from tenacity import retry, stop_after_attempt
import simplejson as json
from selenium import webdriver

from utils import ArticleMeta, NewsCrawler

In [ ]:
class EttodayNewsCrawler(NewsCrawler):
    
    def date_to_newslist_url(self, date):
        
        return "https://www.ettoday.net/news/news-list-%s-0.htm" % date
    
    def unfold_abbr_newslink(self, url):
        
        return "https://www.ettoday.net" + url
    
    def newslink_generator(self):
        
        ettoday_date_form = re.compile("[0-9]{4}/[0-9]{2}/[0-9]{2} [0-9]{2}:[0-9]{2}")
        
        for date in self.date_generator():
            newslist_url = self.date_to_newslist_url(date)
            newslist = self.get_bsObj_scroll_down(newslist_url)
            news_datetimes = newslist.find_all('span', class_='date', text=ettoday_date_form)

            for news_datetime in news_datetimes:
                newslinks = news_datetime.parent.a['href']
                yield newslinks
    
    @retry(stop=stop_after_attempt(0),
           retry_error_callback=lambda x: None)
    def parse_category(self, newspage):
        return newspage.findAll('a', itemprop='item')[1].span.text
    
    @retry(stop=stop_after_attempt(0),
           retry_error_callback=lambda x: None)
    def parse_title(self, newspage):
        return newspage.find('h1', class_='title', itemprop='headline').text
    
    @retry(stop=stop_after_attempt(0),
           retry_error_callback=lambda x: None)
    def parse_article(self, newspage):
        paragraphs = []
        for paragraph in newspage.find('article').find_all('p'):
            if paragraph.findChild():
                continue
            else:
                paragraphs.append(paragraph.text)
        return '\n'.join(paragraphs)
    
    @retry(stop=stop_after_attempt(0),
           retry_error_callback=lambda x: (None, None))
    def parse_date_time(self, newspage):
        # workaround
        datetime_string = newspage.find('time')['datetime'][:-6]
        dt = datetime.datetime.strptime(datetime_string, "%Y-%m-%dT%H:%M:%S")
        return str(dt.date()), str(dt.time())
    
    def is_valid_newspage(self, bsObj):
        
        if bsObj is None:
            return False
        
        try:
            if bsObj.find('em').text == '404錯誤':
                return False
            else:
                return True
        except:
            return True
    
    def saved_filename(self, url):
        return url.split('/')[-1].split('.')[0] + '.json'
    
    def crawl_and_save(self):
        
        for newslink in self.newslink_generator():
            article = self.get_page_attribute_from_link(self.unfold_abbr_newslink(newslink))
            
            if article is None:
                continue

            self.save_article_meta(article)

In [ ]:
ettoday_crawler = EttodayNewsCrawler(
    output_dir='../news/ettoday',
    total_days=1,
    start_date=datetime.date(year=2019, month=7, day=29)
)

In [ ]:
ettoday_crawler.crawl_and_save()

In [ ]:
article = ettoday_crawler.get_page_attribute_from_link('https://www.ettoday.net/news/20190615/1467832.htm')